In [57]:
import numpy as np
import netCDF4
import h5netcdf
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import pickle
from os.path import join
sys.path.append('/home/samuel.varga/projects/deep_learning/')
sys.path.append('/home/samuel.varga/python_packages/fronts/')
from deep_learning.training_utils import load_rotation, convert_to_tf, resize_neural_net
from deep_learning.deep_networks import create_U_net_classifier_2D
from custom_losses import brier_skill_score, critical_success_index, fractions_skill_score
from custom_losses import brier_skill_score_metric, critical_success_index_metric, fractions_skill_score_metric

In [58]:
from keras import backend as K
import gc
from numba import cuda
outdir='/work/samuel.varga/projects/2to6_hr_severe_wx/DEEP_LEARNING/models/'

# Binary Classification

In [59]:
batch_size=2048
target_column='any_severe__36km'
rotation=3
X_train, y_train, mean, variance = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__training_data.nc'), rotation, target_column)
X_val, y_val = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__validation_data.nc'), rotation, target_column)
X_test, y_test = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__testing_data.nc'), None, target_column)
print(np.shape(X_train))
print(np.shape(y_train))

#Convert to tf dataset
train_ds = convert_to_tf((X_train,np.expand_dims(y_train, axis=-1)), batch_size)
val_ds = convert_to_tf((X_val, np.expand_dims(y_val, axis=-1)), batch_size)
test_ds=convert_to_tf((X_test[None,:,:,:], np.expand_dims(y_test[None,:,:,:], axis=-1)))

Training path detected - loading scaling
(6290, 16, 16, 63)
(6290, 16, 16)


In [60]:
#U-net architectural parameters
i=4
conv_filters=[i*n for n in [32,64,128,256]]
conv_size=[2,1,2,1]
max_pool=[2,2,2,2]
conv_layers =[{'filters': f, 'kernel_size': (s), 'pool_size': (p), 'strides': (p)} if p > 1
                   else {'filters': f, 'kernel_size': (s), 'pool_size': None, 'strides': None}
                   for s, f, p, in zip(conv_size, conv_filters, max_pool)]
args={'lrate':1e-4, 'loss':'binary_crossentropy','activation_conv':'relu','activation_out':'sigmoid',
     'p_spatial_dropout':0.1, 'filters':conv_filters, 'size':conv_size, 'pool':max_pool, 'shape':(16,16),
     'rotation':rotation,'target_column':target_column, 'i':i}
thresholds= [0.15]
metrics=[tf.keras.metrics.SparseCategoricalAccuracy(), tf.keras.metrics.MeanSquaredError(name='Brier score'),
    tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.AUC(name='prc', curve='PR'),     
         tf.keras.metrics.FalseNegatives(thresholds=thresholds, name='FN'), tf.keras.metrics.FalsePositives(thresholds=thresholds, name='FP'),
         tf.keras.metrics.MeanAbsoluteError(name='MAE'), tf.keras.metrics.TrueNegatives(thresholds=thresholds, name='TN'),
         tf.keras.metrics.TruePositives(thresholds=thresholds, name='TP'), brier_skill_score_metric(), critical_success_index_metric()]

In [61]:
#Create U-net
u_net = create_U_net_classifier_2D(image_size=args['shape'], nchannels=63, n_classes=1, conv_layers=conv_layers, p_spatial_dropout=args['p_spatial_dropout'], metrics=metrics,
                               lrate=args['lrate'], loss=args['loss'], activation_conv=args['activation_conv'], activation_out='sigmoid',
                                      normalization=(mean, variance))

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 16, 16, 63)  0           []                               
                                ]                                                                 
                                                                                                  
 normalization_7 (Normalization  (None, 16, 16, 63)  0           ['Input[0][0]']                  
 )                                                                                                
                                                                                                  
 Encode_0_0 (Conv2D)            (None, 16, 16, 128)  32384       ['normalization_7[0][0]']        
                                                                                            

 De_Sp_Dr_2_1 (SpatialDropout2D  (None, 4, 4, 512)   0           ['Decode_2_1[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_2_0 (Conv2D)            (None, 4, 4, 512)    1049088     ['De_Sp_Dr_2_1[0][0]']           
                                                                                                  
 De_Sp_Dr_2_0 (SpatialDropout2D  (None, 4, 4, 512)   0           ['Decode_2_0[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_Upsample_1 (UpSampling2  (None, 8, 8, 512)   0           ['De_Sp_Dr_2_0[0][0]']           
 D)                                                                                               
          

In [62]:
#Callbacks
early_stopping_cb =keras.callbacks.EarlyStopping(patience=25, restore_best_weights=True,
                                                min_delta=0.01, monitor='val_loss')
tensorboard_cb= keras.callbacks.TensorBoard(log_dir=join(outdir,'logs'), histogram_freq=1)

In [ ]:
#Learn the model
history = u_net.fit(train_ds, epochs=100, verbose=True, validation_data = val_ds,
    callbacks=[early_stopping_cb, tensorboard_cb])

Epoch 1/100


2024-02-14 18:01:56.448239: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_7/En_Sp_Dr_0_0/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


4/4 [==============================] - 3s 372ms/step - loss: 0.6472 - sparse_categorical_accuracy: 0.9454 - Brier score: 0.2270 - auc: 0.5064 - prc: 0.0990 - FN: 846.0000 - FP: 1521871.0000 - MAE: 0.4726 - TN: 492.0000 - TP: 87031.0000 - bss: 0.7858 - csi: 0.0446 - val_loss: 0.5227 - val_sparse_categorical_accuracy: 0.9331 - val_Brier score: 0.1632 - val_auc: 0.2749 - val_prc: 0.0441 - val_FN: 5410.0000 - val_FP: 372187.0000 - val_MAE: 0.3846 - val_TN: 7604.0000 - val_TP: 21839.0000 - val_bss: 0.8368 - val_csi: 0.0445
Epoch 2/100
4/4 [==============================] - 1s 266ms/step - loss: 0.4610 - sparse_categorical_accuracy: 0.9454 - Brier score: 0.1317 - auc: 0.3250 - prc: 0.0382 - FN: 32408.0000 - FP: 1360082.0000 - MAE: 0.3268 - TN: 162281.0000 - TP: 55469.0000 - bss: 0.8803 - csi: 0.0291 - val_loss: 0.4542 - val_sparse_categorical_accuracy: 0.9331 - val_Brier score: 0.0991 - val_auc: 0.2966 - val_prc: 0.0455 - val_FN: 20307.0000 - val_FP: 176928.0000 - val_MAE: 0.2216 - val_TN: 2

In [ ]:
def save_results(args, u_net, history, train_ds, val_ds, test_ds):
    fbase = f"{args['target_column']}_Rot_{args['rotation']}_{args['shape'][0]}_{args['shape'][1]}_lrate_{args['lrate']}_spatial_dropout_{args['p_spatial_dropout']}_i_{args['i']}_filters_{args['filters']}_size_{args['size']}_pool_{args['pool']}_loss_{args['loss']}"
    results = {}
    results['args'] = args
    #results['predict_val'] = u_net.predict(val_ds)
    results['predict_val_eval'] = u_net.evaluate(val_ds, return_dict=True)
    results['predict_val'] = u_net.predict(val_ds)
    print(results['predict_val_eval'])

    if test_ds is not None:
        results['predict_test']=u_net.predict(test_ds)
        results['predict_test_eval']=u_net.evaluate(test_ds, return_dict=True)

    #results['predict_train']=u_net.predict(train_ds)
    results['predict_train_eval']=u_net.evaluate(train_ds, return_dict=True)
    results['history']=history.history
    results['fname_base']=fbase

    #Save results
    with open(join(outdir, f'{fbase}_results.pkl'),'wb') as fp:
        pickle.dump(results, fp)

    #save model
    if False:
        u_net.save(join(join(outdir, 'models'), f'{fbase}_model'))
    #print(np.max(results['predict_test']))
    #print(np.mean(results['predict_test']))
    print(fbase)
    return None
save_results(args, u_net, history,  train_ds, val_ds, test_ds)